In [ ]:
from traitlets import Unicode, Bool, validate, TraitError, observe
from ipywidgets import DOMWidget, register
import time
import threading
import json

In [ ]:
def py2js_send_five_messages(widget):
    def func():
        for i in range(3):
            widget.pyjs_channel = f"Hello from Python: {i+1}"
            time.sleep(1)
    return func

jspy_message = ""

@register
class WebVisualizer(DOMWidget):
    _view_name = Unicode('WebVisualizerView').tag(sync=True)
    _view_module = Unicode('email_widget').tag(sync=True)
    _view_module_version = Unicode('0.1.0').tag(sync=True)
    
    # Attributes
    pyjs_channel = Unicode("Empty pyjs_channel.", help="Python->JS message channel.").tag(sync=True)
    jspy_channel = Unicode("Empty jspy_channel.", help="JS->Python message channel.").tag(sync=True)
    
    #def pyjs_send(self, message):
    #    # message = json.dumps({
    #    #   "call_id"    : "my_id_00",
    #    #   "json_result": "my_result_00"
    #    # })
    #    #
    #    # self.pyjs_channel = json.dumps({
    #    #   "my_id_00": "my_result_00",
    #    #   "my_id_01": "my_result_01",
    #    #   "my_id_02": "my_result_02"
    #    # })
    #    #
    #    # pyjs_send() never clears self.pyjs_channel, instead, it adds a dict
    #    # entry to self.pyjs_channel.
    #    #
    #    # self.pyjs_channel is cleared at javascript's render() call.
    #
    #    # Parse input
    #    message_dict = json.loads(message)
    #    if "call_id" not in message_dict:
    #        raise ValueError(f"pyjs_send call_id not in message: {message}")
    #    call_id = message_dict["call_id"]
    #    if "json_result" not in message_dict:
    #        raise ValueError(f"json_result call_id not in message: {message}")
    #    json_result = message_dict["json_result"]
    #
    #    # Insert new entry to channel_dict
    #    channel_dict = json.loads(self.pyjs_channel)
    #    channel_dict[call_id] = json_result
    #
    #    self.pyjs_channel = json.dumps(channel_dict)
        
    def call_http_request(self, entry_point, query_string, data):
        return f"Called Http Request: {entry_point}, {query_string}, {data}!"

    @observe('jspy_channel')
    def on_jspy_message(self, change):
        
        # self.result_map = {"0": "result0", "1": "result1"};
        if not hasattr(self, "result_map"):
            self.result_map = dict()

        jspy_message = change["new"]
        print(f"js->py message received: {jspy_message}")
        new_call = False
        try:
            jspy_requests = json.loads(jspy_message);
            print(f"!!! jspy_message: {jspy_message}")
            print(f"!!! jspy_requests: {jspy_requests}")
            print(f"!!! type(jspy_requests): {type(jspy_requests)}")
            
            for call_id, payload in jspy_requests.items():
                print(f"!!! ONE call_id: {payload}")
                print(f"!!! ONE payload: {payload}")
                if "func" not in payload or payload["func"] != "call_http_request":
                    raise ValueError(f"Invalid jspy function: {jspy_requests}")
                if "args" not in payload or len(payload["args"]) != 3:
                    raise ValueError(
                        f"Invalid jspy function arguments: {jspy_requests}")
                    
                # Check if already in result
                if not call_id in self.result_map:
                    json_result = self.call_http_request(payload["args"][0],
                                                         payload["args"][1],
                                                         payload["args"][2])
                    self.result_map[call_id] = json_result
                    new_call = True
        except:
            print(
                f"js->py message is not a function call, ignored: {jspy_message}"
            )
        else:
            print(f"py->js sending: {self.result_map}")
            self.pyjs_channel = json.dumps(self.result_map)

In [ ]:
%%javascript

require.undef('email_widget');

define('email_widget', ["@jupyter-widgets/base"], function(widgets) {
    var WebVisualizerView = widgets.DOMWidgetView.extend({
        sleep: function(time_ms) {
            return new Promise((resolve) => setTimeout(resolve, time_ms));
        },
        
        jspy_send: function(message) {
            this.model.set("jspy_channel", message);
            this.touch();
        },
        
        // args must be all strings.
        // TODO: kwargs and sanity check
        callPython: async function (func, args = []) {
            var callId = this.callId.toString();
            this.callId++;
            var message = {
              func: func,
              args: args,
              call_id: callId,
            };
            
            // Append message to current jspy_channel
            var jspyChannel = this.model.get("jspy_channel");
            var jspyChannelObj = JSON.parse(jspyChannel);
            jspyChannelObj[callId] = message;
            jspyChannel = JSON.stringify(jspyChannelObj);            
            this.jspy_send(jspyChannel);
            
            var count = 0;
            while (!this.callResultReady(callId)) {
              console.log("callPython await, id: " + callId + ", count: " + count++);
              await this.sleep(1000);
            }
            var json_result = this.extractCallResult(callId);
            console.log(
              "callPython await done, id:",
              callId,
              "json_result:",
              json_result
            );
            return json_result;
        },
        
        render: function() {
            this.model.set("pyjs_channel", "{}");
            this.model.set("jspy_channel", "{}");
            this.touch();
            
            // Python call registry
            this.callId = 0;
            
            this.email_input = document.createElement('p');
            this.el.appendChild(this.email_input);
            
            // Listen for py->js message.
            // this.model.on('change:pyjs_channel', this.on_pyjs_message, this);
            
            // Send js->py message for testing.
            this.callPython("call_http_request", 
                            ["my_entry_point", "my_query_string", "my_data"]).then(
                (result) => {console.log("callPython.then()", result)}
            );
            
            this.callPython("call_http_request", 
                            ["my_entry_point", "my_query_string", "my_data"]).then(
                (result) => {console.log("callPython.then()", result)}
            );
        },
        
        callResultReady: function (callId) {
          var pyjs_channel = this.model.get("pyjs_channel");
          console.log("Current pyjs_channel:", pyjs_channel);
          var callResultMap = JSON.parse(this.model.get("pyjs_channel"));
          return callId in callResultMap;
        },

        extractCallResult: function (callId) {
          if (!this.callResultReady(callId)) {
            throw "extractCallResult not ready yet.";
          }
          var callResultMap = JSON.parse(this.model.get("pyjs_channel"));
          return callResultMap[callId];
        },
        
        // on_pyjs_message: function() {
        //     var message = this.model.get('pyjs_channel');
        //     message = "pyjs_message received: " + message;
        //     this.email_input.innerText = this.email_input.innerText 
        //                                + "\n" 
        //                                + message;
        //     this.new_pyjs_message = true;
        // },
        
    });
    
    return {
        WebVisualizerView: WebVisualizerView
    }
});

In [ ]:
visualizer = WebVisualizer()
visualizer